In [1]:
import pandas as pd
import datetime
import json
import numpy as np
import string
import math
import re
import matplotlib.pyplot as plt
import seaborn as sns

from torch.utils.data import TensorDataset

import torchtext
from torchtext.vocab import Vectors
from torchtext import data, datasets

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from model import QRDQN

# データのロード

In [2]:
for i, date in enumerate(range(2011, 2019)):
    tmp = pd.read_csv('./data/news/' + str(date) + '.csv', encoding='cp932')
    tmp = tmp[tmp['Company_IDs(TSE)'] == '7203']
    tmp = tmp[['Time_Stamp_Original(JST)', 
                        'Company_Code(TSE)', 
                        'Headline', 
                        'News_Source',
                        'Company_Relevance', 
                        'Keyword_Article']]

    # 欠損除去
    tmp = tmp[~tmp["Keyword_Article"].isnull()]

    # タグ除去
    tmp = tmp[(tmp['News_Source'] == '日経') | 
                        (tmp['News_Source'] == 'ＮＱＮ') |
                        (tmp['News_Source'] == 'ＱＵＩＣＫ') | 
                        (tmp['News_Source'] == 'Ｒ＆Ｉ')]

    tmp.index = pd.to_datetime(tmp["Time_Stamp_Original(JST)"])
    tmp = tmp.drop("Time_Stamp_Original(JST)", axis=1)
    
    if i == 0:
        df1 = tmp.copy()
    else:
        df1 = pd.concat([df1, tmp])

# インデックスを設定

In [3]:
def norm_time(x):
    if x.hour > 15:
        return x + datetime.timedelta(days=1)
    return x

time = pd.to_datetime(df1.index.values)
df1.index = df1.index.map(norm_time)
df1.index = df1.index.date

# 株価を挿入する

In [4]:
# 株価を取り出す
df2 = pd.read_csv('./data/stock_price/7203.csv', index_col=0)
df2.index = pd.to_datetime(df2['date'])
df2.index = df2.index.date
df2 = df2.drop(['date'], axis=1)
df2.head(10)

,adj_close
2011-01-04,3265.0
2011-01-05,3295.0
2011-01-06,3380.0
2011-01-07,3455.0
2011-01-11,3455.0
2011-01-12,3500.0
2011-01-13,3535.0
2011-01-14,3550.0
2011-01-17,3500.0
2011-01-18,3510.0


# 時系列をくっつける

In [5]:
df3 = pd.concat([df1,df2], axis=1, join_axes=[df1.index])
df3['price'] = np.round(df2.pct_change().shift(-1) * 100, 3)
df3['Keyword_Article'] = \
    df3.groupby(level=0).apply(lambda x: ':<pad>:'.join(list(x['Keyword_Article'])))
df3 = df3.dropna()

df3 = df3[~df3.duplicated(subset=['Keyword_Article'])]

/home/shogo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  """Entry point for launching an IPython kernel.


In [6]:
df3.head()

,Company_Code(TSE),Headline,News_Source,Company_Relevance,Keyword_Article,adj_close,price
2011-01-04,7203.0,<日経>◇次世代車の研究開発 名大に国内最大拠点,日経,38,安全:環境:負荷:開発:目指す:開所式:研究拠点:効率:簡素化:次世代:電気自動車:電気:幅...,3265.0,0.919
2011-01-05,7203.0,<日経>◇12月の中国新車販売、トヨタが単月で過去最高,日経,100,北京:中国:１２月:新車販売台数:前年同月比:増:過去最高:制限:受け:全国:各地:乗用車:...,3295.0,2.580
2011-01-06,7203.0,<NQN>◇トヨタ社長「今年は後半に晴れ間」 為替は１ドル＝90円を期待,ＮＱＮ,100,豊田:見通し:販売:エコカー補助金:安定的:伸び:株価:為替:水準:日経平均株価:最低:ライ...,3380.0,2.219
2011-01-07,7203.0,<日経>◇福岡県、自動車の技術者育成へ新組織 年内、中小向け,日経,37,自動車産業:強化:福岡:先端:設置:方針:技術:調査:ニーズ:カリキュラム:大学:受け:生産...,3455.0,0.000
2011-01-11,7203.0,<日経>◇トヨタ、米ミシガン州に安全研究センター新設,日経,100,先進:安全:子供:高齢者:事故:向上:目指す:米国:大規模:リコール:回収:問題:開催:豊田...,3455.0,1.302


# csvファイルに保存する

In [7]:
train_date = 2015
test_date = 2017

In [8]:
df4 = pd.concat([df3[['Keyword_Article', 'price']].rename(
                                      columns={'Keyword_Article': 'state', 'price': 'reward'}),
                               df3[['Keyword_Article']].shift(-1).rename(
                                      columns={'Keyword_Article': 'next_state'})], axis=1).dropna()
df4 = df4[['state', 'next_state', 'reward']]

date_year = df4.index.map(lambda x: x.year)

In [9]:
df4[date_year <= train_date].to_csv(
        './data/news/text_train.tsv',
        header=None,
        index=None,
        sep='\t')

In [10]:
df4[(train_date < date_year) & (date_year < test_date)].to_csv(
        './data/news/text_val.tsv',
        header=None,
        index=None,
        sep='\t')

In [11]:
df4[test_date <= date_year].to_csv(
        './data/news/text_test.tsv',
        header=None,
        index=None,
        sep='\t')

# Dataの作成

In [12]:
# 前処理
def preprocessing_text(text):
    # カンマ、ピリオド以外の記号をスペースに置換
    for p in string.punctuation:
        if (p == ".") or (p == ",") or (p == ":") or (p == "<")or (p == ">"):
            continue
        else:
            text = text.replace(p, " ")

    # ピリオドなどの前後にはスペースを入れておく
    text = text.replace(".", " . ")
    text = text.replace(",", " , ")
    text = re.sub(r'[0-9 ０-９]', '0', text)
    
    return text

# 分かち書き（今回はデータが英語で、簡易的にスペースで区切る）
def tokenizer_punctuation(text):
    return text.strip().split(':')

# 前処理と分かち書きをまとめた関数を定義
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)
    ret = tokenizer_punctuation(text)
    return ret

In [95]:
max_length = 256
batch_size = 32

# 読み込んだ内容に対して行う処理を定義
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, 
                            use_vocab=True,
                            lower=True, include_lengths=True, batch_first=True, fix_length=max_length, 
                            init_token="<cls>", eos_token="<eos>")
LABEL = torchtext.data.Field(sequential=False, use_vocab=False, dtype=torch.float)

In [96]:
train_ds = torchtext.data.TabularDataset.splits(
    path='./data/news', train='text_train.tsv',
    format='tsv',
    fields=[('Text1', TEXT), ('Text2', TEXT), ('Label', LABEL)])
train_ds = train_ds[0]

# japanese_fasttext_vectors = Vectors(name='./data/news/cc.ja.300.vec')
TEXT.build_vocab(train_ds, 
#                  vectors=japanese_fasttext_vectors,
                 min_freq=10)
TEXT.vocab.freqs

train_dl = torchtext.data.Iterator(
    train_ds, batch_size=batch_size, train=True)

In [97]:
batch = next(iter(train_dl))
print(batch.Text1)
print(batch.Text2)
print(batch.Label)

(tensor([[   2,   52,  158,  ...,  150,    1,    3],
        [   2,  480,  311,  ...,    1,    1,    1],
        [   2,  335,  117,  ...,    1,    1,    1],
        ...,
        [   2,    4,  145,  ...,    1,    1,    1],
        [   2,  118,  179,  ...,    1,    1,    1],
        [   2,  862, 1234,  ...,    1,    1,    1]]), tensor([256, 214, 110, 256, 114, 221, 136,  18, 117, 244,  31, 256,  69,  34,
         64,  75,  15, 256, 102,  15,  61,  91,  15,  55,  25,  49, 102, 244,
        106,  91,  85, 118]))
(tensor([[   2,  147,  196,  ...,    1,    1,    1],
        [   2,  263,  111,  ...,  332,   40,    3],
        [   2,   48,    4,  ...,    1,    1,    1],
        ...,
        [   2,    4,  207,  ...,    1,    1,    1],
        [   2,    0, 1158,  ...,    1,    1,    1],
        [   2,   78,  193,  ...,    1,    1,    1]]), tensor([166, 256, 108,  43,  26, 145, 244,  90, 108,  76,  81, 256,  73,  52,
        256,  45,  58, 201, 239,   5,  17,  98, 118, 107,  60,  59,  80, 256,
  

In [98]:
batch.Text1[0][11]

tensor([   2,    4,  148,  139,  114,    0, 1152,  388, 1358, 2038,  463, 1090,
         600,  290, 1618,  477,  112,  201,    1,    4, 1367,  139,  114,  777,
        1469,  133,   66,  148, 1057, 1257, 1874, 1453,  126,  693,  463,  556,
           0,  178, 1160, 1205,  136,  388, 1008, 1356,  880,  468,  819,  181,
          35,  858, 1825,  497,  505, 1345, 1717,    0,    0,   61,   56,   73,
        1171,   25,   32, 1179,  175,  812, 1326,  824,  346,  112,  201,    1,
           4, 1264,  456,  240,  259,   13, 1444,  271,    0,   16,  842,  112,
         128,   93,  185,   32,   25,  811,    6,   24,   21, 1676,  211,   14,
          41,  115,  341, 1101,   38,    7, 1298,  617,  394,    5,  393,  716,
         622,  380,  276,   48,   66,  699,  122,    0,  359,  614,  453,  119,
         114,  201,    1, 1427, 1526,    0,  182,  218,  123,   77, 1723,  710,
           1,    4,   76,  163,  339, 1608,    5, 1190,    4,   29, 1104,   46,
         207,   25,   41,   14,   60,  3

# モデル構築

In [99]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [100]:
VOCAB_SIZE = len(TEXT.vocab.freqs)
EMBEDDING_DIM = 300
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
PAD_IDX = 1
GAMMA = 0.99

In [101]:
class DQN(nn.Module):
    def __init__(self, text_embedding_vector,vocab_size, embedding_dim, 
                    n_filters, filter_sizes, pad_idx,
                    d_model=300, num_actions=2, quantiles=51):
        super().__init__()

        self.num_actions = num_actions

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)

        self.convs = nn.ModuleList([
            nn.Conv2d(in_channels=1,
                      out_channels=n_filters,
                      kernel_size=(fs, embedding_dim))
            for fs in filter_sizes
        ])

        self.fc = nn.Linear(len(filter_sizes) * n_filters, self.num_actions)

    def forward(self, text):
        embedded = self.embedding(text)    # [batch size, sent len, emb dim]

        embedded = embedded.unsqueeze(1)   # [batch size, 1, sent len, emb dim]

        h = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]   # [batch size, n_filters, sent len - filter_sizes[n] + 1]

        h = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in h]

        h = torch.cat(h, dim=1)

        h = self.fc(h)

        return h


In [22]:
model = DQN(TEXT.vocab.vectors, VOCAB_SIZE, EMBEDDING_DIM, N_FILTERS,
                        FILTER_SIZES, PAD_IDX)

target_model = DQN(TEXT.vocab.vectors, VOCAB_SIZE, EMBEDDING_DIM, N_FILTERS,
                        FILTER_SIZES, PAD_IDX)

model = model.to(device)
target_model = tarmax_length = 1000
batch_size = 32

# 読み込んだ内容に対して行う処理を定義
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, 
                            use_vocab=True,
                            lower=True, include_lengths=True, batch_first=True, fix_length=max_length, 
                            init_token="<cls>", eos_token="<eos>")
LABEL = torchtext.data.Field(sequential=False, use_vocab=False, dtype=torch.float)get_model.to(device)

target_model.load_state_dict(model.state_dict())

<All keys matched successfully>

# 最適化

In [23]:
# 最適化手法
learning_rate = 2.5e-4
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


criterion = nn.BCEWithLogitsLoss()
criterion = criterion.to(device)

In [24]:
def accuracy(scores, y):    
    correct = (scores == y)
    acc = correct.sum() / len(correct)
    return acc

def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()
    return acc

def huber(x):
        cond = (x.abs() < 1.0).float().detach()
        return 0.5 * x.pow(2) * cond + (x.abs() - 0.5) * (1.0 - cond)

In [62]:
# curr_q
states = batch.Text1[0].to(device)
next_states = batch.Text2[0].to(device)
rewards = batch.Label.to(device)

with torch.no_grad():
    actions = torch.argmax(model(states), 1)
    actions = torch.where(torch.randn(len(states)).to(device) >= 0, 
                          actions, 
                          (actions + 1) % 2)

    selected_actions = actions.detach().cpu().numpy()


# epi_rewards.append((selected_actions * rewards.detach().cpu().numpy()).sum())
# neutrals.append(len(selected_actions[selected_actions == 0]))
# buys.append(len(selected_actions[selected_actions == 1]))

actions = actions.view(-1, 1)
curr_q = model(states).gather(1, actions).squeeze(dim=1)

# target_q
with torch.no_grad():

    next_actions = torch.argmax(model(next_states), 1).view(-1, 1)

    next_q = target_model(next_states).gather(1, next_actions)
    target_q = rewards.view(-1, 1) + (GAMMA * next_q)

loss = torch.mean((target_q - curr_q)**2)

# Optimize the model
optimizer.zero_grad()
loss.backward()
for param in model.parameters():
    param.grad.data.clamp_(-1, 1)
optimizer.step()


In [64]:
num_epochs = 100
TARGET_UPDATE_FREQ = 10
# dataloaders_dict = {'train': train_dl, 'val':val_dl}
dataloaders_dict = {'train': train_dl}

print('----start----')

torch.backends.cudnn.benchmark = True

for epoch in range(num_epochs):
    epi_rewards = []
    neutrals = []
    buys = []
    
    # update target_model
    if epoch % TARGET_UPDATE_FREQ == 0:
        target_model.load_state_dict(model.state_dict())
    
    for batch in (dataloaders_dict['train']):      
        # curr_q
        states = batch.Text1[0].to(device)
        next_states = batch.Text2[0].to(device)
        rewards = batch.Label.to(device)
    
        with torch.no_grad():
            actions = torch.argmax(model(states), 1)
            actions = torch.where(torch.randn(len(states)).to(device) >= 0, 
                                  actions, 
                                  (actions + 1) % 2)

            selected_actions = actions.detach().cpu().numpy()
            actions = actions.view(-1, 1)

        epi_rewards.append((selected_actions * rewards.detach().cpu().numpy()).sum())
        neutrals.append(len(selected_actions[selected_actions == 0]))
        buys.append(len(selected_actions[selected_actions == 1]))
        
        curr_q = model(states).gather(1, actions).squeeze(dim=1)

        # target_q
        with torch.no_grad():

            next_actions = torch.argmax(model(next_states), 1).view(-1, 1)

            next_q = target_model(next_states).gather(1, next_actions)
            target_q = rewards.view(-1, 1) + (GAMMA * next_q)

        loss = torch.mean((target_q - curr_q)**2)

        # Optimize the model
        optimizer.zero_grad()
        loss.backward()
        for param in model.parameters():
            param.grad.data.clamp_(-1, 1)
        optimizer.step()
    
    print('--------------------')
    print('epoch:', epoch)
    print('loss:', loss.item())
    print('epi_reward:', sum(epi_rewards))
    print('neutrals:', sum(neutrals), '  buys:', sum(buys))

----start----
--------------------
epoch: 0
loss: 2.0674140453338623
epi_reward: 53.54200131818652
neutrals: 490   buys: 529
--------------------
epoch: 1
loss: 4.894399166107178
epi_reward: 27.3910000808537
neutrals: 511   buys: 508
--------------------
epoch: 2
loss: 1.9778602123260498
epi_reward: 36.14900116622448
neutrals: 513   buys: 506
--------------------
epoch: 3
loss: 5.133679389953613
epi_reward: 43.3600003644824
neutrals: 502   buys: 517
--------------------
epoch: 4
loss: 1.8334323167800903
epi_reward: 23.222000082954764
neutrals: 500   buys: 519
--------------------
epoch: 5
loss: 3.417635440826416
epi_reward: 39.153000354766846
neutrals: 515   buys: 504
--------------------
epoch: 6
loss: 5.337892055511475
epi_reward: 38.70099985413253
neutrals: 526   buys: 493
--------------------
epoch: 7
loss: 4.28289794921875
epi_reward: 57.407000264152884
neutrals: 506   buys: 513
--------------------
epoch: 8
loss: 2.911771774291992
epi_reward: 50.74299978837371
neutrals: 500   buy

--------------------
epoch: 74
loss: 2.2340328693389893
epi_reward: 51.07299919426441
neutrals: 495   buys: 524
--------------------
epoch: 75
loss: 3.8179001808166504
epi_reward: 75.05400026403368
neutrals: 507   buys: 512
--------------------
epoch: 76
loss: 5.913333892822266
epi_reward: 43.88199939019978
neutrals: 526   buys: 493
--------------------
epoch: 77
loss: 3.699885845184326
epi_reward: 43.87299994751811
neutrals: 477   buys: 542
--------------------
epoch: 78
loss: 2.6545627117156982
epi_reward: 49.22800051793456
neutrals: 527   buys: 492
--------------------
epoch: 79
loss: 2.3813862800598145
epi_reward: 55.07000015862286
neutrals: 518   buys: 501
--------------------
epoch: 80
loss: 1.7790753841400146
epi_reward: 55.151000302284956
neutrals: 505   buys: 514
--------------------
epoch: 81
loss: 1.8370882272720337
epi_reward: 10.405999476090074
neutrals: 541   buys: 478
--------------------
epoch: 82
loss: 4.761754035949707
epi_reward: 16.193000299856067
neutrals: 513   bu

# 描画

In [65]:
batch = next(iter(train_dl))
states = batch.Text1[0].to(device)
next_states = batch.Text2[0].to(device)
rewards = batch.Label.to(device)

In [83]:
# state = states[4]
actions = torch.argmax(model(states), 1)
# dist_action = actions[0].cpu().detach().numpy()
# # sns.distplot(dist_action[0], bins=51, color='red')
# sns.distplot(dist_action[1], bins=10, color='blue')
# plt.show()

In [84]:
actions

tensor([1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
        0, 0, 0, 0, 1, 0, 0, 1], device='cuda:0')